In [1]:
import os 
import datetime
from pathlib import Path
from dotenv import load_dotenv, find_dotenv

basepath = Path(os.getcwd())
# make sure your working directory is the repository root.
if basepath.name != "idp-radio-1":
    os.chdir(basepath.parent.parent.parent)
load_dotenv(find_dotenv())

%load_ext autoreload
%autoreload 2
os.getcwd()

'/srv/idp-radio-1'

In [2]:
import os 
import tensorflow as tf
from pathlib import Path

In [ ]:
config


In [3]:
# Run this before loading other dependencies, otherwise they might occupy memory on gpu 0 by default and it will stay that way

# Specify which GPU(s) to use
os.environ["CUDA_VISIBLE_DEVICES"] = "1,0"  # Or 2, 3, etc. other than 0

config = tf.compat.v1.ConfigProto(device_count={'GPU': 2}, allow_soft_placement=True, log_device_placement=True)
#config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 2.0
tf.compat.v1.Session(config=config)


Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:1 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:42:00.0, compute capability: 7.5
/job:localhost/replica:0/task:0/device:GPU:1 -> device: 1, name: GeForce RTX 2080 Ti, pci bus id: 0000:08:00.0, compute capability: 7.5



In [4]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.applications.resnet_v2 import ResNet101V2
from src.architectures.simple.simple_base import SimpleBaseArchitecture
from src.architectures.adv.guendel19 import densenet

from src.architectures.benchmarks.benchmark import Benchmark, Experiment
from tensorflow.keras.optimizers import Adam
from src.architectures.benchmarks.benchmark_definitions import Chexpert_Benchmark, Chestxray14_Benchmark, simple_architecture_experiment, generate_benchmarks, METRICS, CHEXPERT_COLUMNS, CHESTXRAY14_COLUMNS
from src.metrics.metrics import F2Score
from src.metrics.losses import WeightedBinaryCrossentropy, compute_class_weight

In [ ]:
CHEXPERT_BENCHMARKS, CHESTXRAY14_BENCHMARKS = generate_benchmarks (batch_sizes={"Medium": 32}, epoch_sizes= {"Dev": 5,}, crop={"C0": False}, name_suffix="",)



In [5]:
CHEXPERT_BENCHMARKS.keys() 

NameError: name 'CHEXPERT_BENCHMARKS' is not defined

In [ ]:
#CHESTXRAY14_BENCHMARKS.keys()

In [ ]:
#[m.name for m in METRICS]

In [ ]:
#CHEXPERT_COLUMNS

In [2]:
!remote_access/get_tunnels.sh

URLs open for the following services:
[('tensorboard', 'http://16c3b9057210.ngrok.io'), ('jupyterlab', 'http://90a52a2649d5.ngrok.io')]


In [ ]:
chexpert_exp = simple_architecture_experiment(CHEXPERT_BENCHMARKS["BCE_E5_B32_C0"], DenseNet121, CHEXPERT_COLUMNS)

In [ ]:
#chexpert_exp.benchmark.as_dict()

In [ ]:
#chexpert_exp.run()

In [5]:
reduced_columns = ['Enlarged Cardiomediastinum',
                    'Cardiomegaly',
                    'Lung Opacity',
                    'Lung Lesion',
                    'Edema',
                    'Consolidation',
                    'Pneumonia',
                    'Atelectasis',
                    'Pneumothorax',
                    'Pleural Effusion',
                    'Pleural Other',
                    'Fracture']
len(reduced_columns)

12

In [6]:
chexpert_benchmarks, _ = generate_benchmarks(path = Path(os.environ.get("CHEXPERT_DATASET_DIRECTORY")),
                                                     name_suffix="_investigate",
                                                     classes=reduced_columns,
                                                     train_labels = "nofinding_train.csv",
                                                     nan_replacement = -1, #float("NaN"),
                                                     batch_sizes = {"b": 32},
                                                     epoch_sizes = {"e": 3},
                                                     dim=(320, 320),
                                                     crop = {"C0": False},
                                                     split_seed = 6122156, )

Chestxray_BCE_E3_B32_C0_N12_investigate could not be created
Chestxray_WBCE_E3_B32_C0_N12_investigate could not be created
Chestxray_CWBCE_E3_B32_C0_N12_investigate could not be created


In [7]:
chexpert_benchmarks.keys()

dict_keys(['BCE_E3_B32_C0_N12_investigate', 'WBCE_E3_B32_C0_N12_investigate', 'CWBCE_E3_B32_C0_N12_investigate'])

In [8]:
chexpert_exp = simple_architecture_experiment(chexpert_benchmarks["CWBCE_E3_B32_C0_N12_investigate"], DenseNet121, reduced_columns)

29089792/29084464 [==============================] - 12s 0us/step


In [9]:
chexpert_exp.benchmark.as_dict()

{'benchmark_name': 'Chexpert_CWBCE_E3_B32_C0_N12_investigate',
 'dataset_name': 'chexpert_full',
 'dataset_folder': 'data/chexpert/full',
 'models_dir': 'models',
 'epochs': 3,
 'optimizer': 'Adam',
 'loss': 'weighted_binary_crossentropy',
 'use_class_weights': False,
 'positive_weights': [5.492364406585693,
  2.4167683124542236,
  1.3062094449996948,
  3.689021348953247,
  1.9961391687393188,
  5.711972713470459,
  8.167060852050781,
  2.7001078128814697,
  4.474693775177002,
  1.6808327436447144,
  8.232630729675293,
  3.7335445880889893],
 'negative_weights': [1.222599983215332,
  1.70583176612854,
  4.2657389640808105,
  1.3718825578689575,
  2.003875732421875,
  1.212225317955017,
  1.1395272016525269,
  1.588197946548462,
  1.2877951860427856,
  2.468789577484131,
  1.1382622718811035,
  1.3658254146575928],
 'metrics': ['auc',
  'precision',
  'recall',
  'f2_score',
  'binary_accuracy',
  'accuracy_enlarged_cardiomediastinum',
  'accuracy_cardiomegaly',
  'accuracy_lung_opacity

In [ ]:
#[(m, m.name) for m in chexpert_exp.benchmark.metrics]

In [ ]:
chexpert_exp.run()

Epoch 1/3
4447/4447 [==============================] - ETA: 0s - loss: 1.1463 - auc: 0.5811 - precision: 0.1692 - recall: 0.7923 - f2_score: 0.4562 - binary_accuracy: 0.4245 - accuracy_enlarged_cardiomediastinum: 0.3299 - accuracy_cardiomegaly: 0.4382 - accuracy_lung_opacity: 0.6224 - accuracy_lung_lesion: 0.2760 - accuracy_edema: 0.5887 - accuracy_consolidation: 0.4545 - accuracy_pneumonia: 0.3398 - accuracy_atelectasis: 0.3940 - accuracy_pneumothorax: 0.4426 - accuracy_pleural_effusion: 0.6767 - accuracy_pleural_other: 0.2839 - accuracy_fracture: 0.2473 - auc_enlarged_cardiomediastinum: 0.4885 - auc_cardiomegaly: 0.6247 - auc_lung_opacity: 0.6738 - auc_lung_lesion: 0.4460 - auc_edema: 0.7337 - auc_consolidation: 0.6507 - auc_pneumonia: 0.5271 - auc_atelectasis: 0.5688 - auc_pneumothorax: 0.6044 - auc_pleural_effusion: 0.7736 - auc_pleural_other: 0.4949 - auc_fracture: 0.3877 - precision_enlarged_cardiomediastinum: 0.0473 - precision_cardiomegaly: 0.1510 - precision_lung_opacity: 0.57

In [15]:
chexpert_exp.evaluate()

100/100 [==============================] - 5s 46ms/step - loss: 1.4067 - auc: 0.5832 - precision: 0.1667 - recall: 0.6129 - f2_score: 0.3992 - binary_accuracy: 0.5542 - accuracy_enlarged_cardiomediastinum: 0.4500 - accuracy_cardiomegaly: 0.4600 - accuracy_lung_opacity: 0.6900 - accuracy_lung_lesion: 0.3900 - accuracy_edema: 0.8000 - accuracy_consolidation: 0.6800 - accuracy_pneumonia: 0.2600 - accuracy_atelectasis: 0.5400 - accuracy_pneumothorax: 0.9400 - accuracy_pleural_effusion: 0.8000 - accuracy_pleural_other: 0.2900 - accuracy_fracture: 0.3500 - auc_enlarged_cardiomediastinum: 0.1753 - auc_cardiomegaly: 0.6236 - auc_lung_opacity: 0.7053 - auc_lung_lesion: 0.3779 - auc_edema: 0.8494 - auc_consolidation: 0.7949 - auc_pneumonia: 0.3832 - auc_atelectasis: 0.5639 - auc_pneumothorax: 0.5891 - auc_pleural_effusion: 0.8543 - auc_pleural_other: 0.6097 - auc_fracture: 0.4719 - precision_enlarged_cardiomediastinum: 0.0000e+00 - precision_cardiomegaly: 0.1228 - precision_lung_opacity: 0.7727 

{'report': '                            precision    recall  f1-score   support\n\nEnlarged Cardiomediastinum       0.04      1.00      0.08         3\n              Cardiomegaly       0.15      0.82      0.25        11\n              Lung Opacity       0.57      0.53      0.55        55\n               Lung Lesion       0.07      0.71      0.12         7\n                     Edema       0.26      0.47      0.33        15\n             Consolidation       0.08      0.43      0.14         7\n                 Pneumonia       0.02      0.67      0.04         3\n               Atelectasis       0.09      0.75      0.16         8\n              Pneumothorax       0.00      0.00      0.00         7\n          Pleural Effusion       0.33      0.43      0.37        35\n             Pleural Other       0.00      0.00      0.00         2\n                  Fracture       0.02      1.00      0.05         2\n\n                 micro avg       0.12      0.52      0.19       155\n                 m

In [ ]:
chexpert_exp.save()

In [11]:
chexpert_exp.train_result.history

{'loss': [1.146277666091919, 1.0943752527236938, 1.0679258108139038],
 'auc': [0.5811475515365601, 0.6099526882171631, 0.6317869424819946],
 'precision': [0.16917496919631958, 0.17395658791065216, 0.17927733063697815],
 'recall': [0.7922576665878296, 0.8157932162284851, 0.8228245377540588],
 'f2_score': [0.4562084376811981, 0.4694058299064636, 0.47896143794059753],
 'binary_accuracy': [0.4245169758796692,
  0.43020153045654297,
  0.4462263882160187],
 'accuracy_enlarged_cardiomediastinum': [0.3299134373664856,
  0.31477680802345276,
  0.311017245054245],
 'accuracy_cardiomegaly': [0.43823084235191345,
  0.47508853673934937,
  0.5189313292503357],
 'accuracy_lung_opacity': [0.6224210262298584,
  0.6301579475402832,
  0.6338824033737183],
 'accuracy_lung_lesion': [0.2759936451911926,
  0.2726065218448639,
  0.2772374749183655],
 'accuracy_edema': [0.5887326002120972, 0.5969333052635193, 0.612358033657074],
 'accuracy_consolidation': [0.4545128643512726,
  0.4501981735229492,
  0.46160331

In [12]:
chexpert_exp.evaluation_result

In [ ]:
print(chexpert_exp.evaluation_result["report"])

In [ ]:
print(chexpert_exp.model_id)

In [ ]:
chexpert_exp.benchmark.name

In [ ]:
"done"

In [ ]:
chexpert_exp.benchmark.testgen.get_labels()

In [ ]:
chexpert_exp.benchmark.testgen.get_labels_nonan()

In [ ]:
predictions = experiment_inceptionnet_chexpert.model.predict(
            experiment_inceptionnet_chexpert.benchmark.testgen, steps=len(experiment_inceptionnet_chexpert.benchmark.testgen), verbose=1)

In [ ]:
true_labels = experiment_inceptionnet_chexpert.benchmark.testgen.get_encoded_labels()

In [ ]:
# model_guendel_chestxray14 = densenet(classes=len(CHESTXRAY14_COLUMNS))
model_guendel_chexpert = densenet(classes=len(CHEXPERT_COLUMNS))

#experiment_guendel_chestxray14 = Experiment(chestxray14_benchmark, model_guendel_chestxray14)
experiment_guendel_chexpert = Experiment(CHEXPERT_BENCHMARKS["WBCE_E10_B32"], model_guendel_chexpert, model_name="test_WBCE_32")



In [ ]:
#experiment_guendel_chestxray14_result =  experiment_guendel_chestxray14.run()

In [ ]:
experiment_guendel_chexpert_result =  experiment_guendel_chexpert.run()

In [ ]:
experiment_guendel_chexpert.evaluate()

In [ ]:
experiment_guendel_chexpert.save(upload=False)

In [ ]:
print("done")

In [ ]:
CHEXPERT_BENCHMARKS["WBCE_E10_B32"].as_dict()

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.losses import Loss
from tensorflow.python.ops.losses import util as tf_losses_util
from tensorflow.python.framework import tensor_util
from tensorflow.python.framework import constant_op
from tensorflow.python.ops import math_ops
from tensorflow.python.ops import clip_ops
from tensorflow.python.keras import backend_config

In [ ]:
weights_pos = tf.ones((4)) * 20.0
weights_neg = tf.ones((4)) * 0.5
y_true = tf.constant([[0, -1, -1, 1], [-1, 0, -1, -1]], dtype=tf.float32)
y_pred = tf.constant([[0.1, 0.2, 0.3, 0.4], [0.5, 0.6, 0.7, 0.8]])
loss = tf.keras.losses.binary_crossentropy(y_true, y_pred)
loss

In [ ]:
y_true = tf.constant(y_true, dtype=tf.float32)
y_true

In [ ]:
y_pred = tf.constant(y_pred)
y_pred

In [ ]:
if tensor_util.is_tensor(y_pred) and tensor_util.is_tensor(y_true):
    y_pred, y_true = tf_losses_util.squeeze_or_expand_dimensions(
        y_pred, y_true)

epsilon_ = constant_op.constant(
     backend_config.epsilon(), dtype=y_pred.dtype.base_dtype)
y_pred = clip_ops.clip_by_value(y_pred, epsilon_, 1. - epsilon_)

In [ ]:
mask = tf.cast(tf.math.greater_equal(y_true, 0), tf.float32)

# Compute cross entropy from probabilities.
bce_pos = y_true * math_ops.log(y_pred + epsilon_)
bce_neg = (1 - y_true) * math_ops.log(1 - y_pred + epsilon_)

bce_pos = tf.math.multiply_no_nan(bce_pos, mask)
bce_neg = tf.math.multiply_no_nan(bce_neg, mask)

bce = bce_pos * weights_pos + \
    bce_neg * weights_neg

loss = tf.reduce_sum(-bce) / tf.reduce_sum(mask)
loss

In [ ]:
tf.reduce_sum(mask)

In [ ]:
bce_neg

In [ ]:
bce

In [ ]:

from tensorflow.keras import backend as K
